In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install evaluate==0.4.3
!pip install llama-cpp-python==0.1.9
!pip install pinecone-client==5.0.1
!pip install langchain_community==0.2.16
!pip install langchain-chroma==0.1.4
!pip install chromadb==0.5.11
!pip install sentence-transformers==3.1.1
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.2/516.2 kB 10.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.1.9-cp310-cp310-linux_x86_64.whl size=176177 sha256=e0fba7ac96a9197002201a1306b9ebd9e874c158c92911b40e49db8e5c641a39
  Stored in directory: /root/.cache/pip/wheels/11/f3/f6/58956cd99c9b4dc6c1065c3d78c7e76ac51df43bb5f9a5fa7b
Successfully built llama-cpp-python
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.

In [ ]:
from langchain_community.document_loaders import PDFMinerLoader, TextLoader, CSVLoader, UnstructuredWordDocumentLoader, UnstructuredHTMLLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from multiprocessing.pool import ThreadPool
from langchain_chroma.vectorstores import Chroma
from langchain.schema import Document
from chromadb.config import Settings
from chromadb import Client
from llama_cpp import Llama
from evaluate import load
from typing import Any
from tqdm import tqdm
from pathlib import Path

In [ ]:
import pandas as pd
import numpy as np
import statistics
import pinecone
import glob
import os

#Лабораторная работа №5

##Declaring constant

In [ ]:
path_to_index = '/content/VDB'
path_to_documents = '/content/drive/MyDrive/ITMO/ML/test_dir' #49960.txt

In [ ]:
# Словарь, сопоставляющий расширения файлов с соответствующими загрузчиками данных и их параметрами
LOADER_MAPPING = {
    ".csv": (CSVLoader, {}),
    ".doc": (UnstructuredWordDocumentLoader, {}),
    ".docx": (UnstructuredWordDocumentLoader, {}),
    ".html": (UnstructuredHTMLLoader, {}),
    ".pdf": (PDFMinerLoader, {}),
    ".txt": (TextLoader, {"encoding": "ISO-8859-1"}),
}

In [ ]:
# Параметры конфигурации для векторного поиска и разделения текста
INDEX_NAME = "VDB"  # Название индекса для хранения векторных представлений
COLLECTION_NAME = "document_collection"
EMBEDDINGS = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"  # Название модели эмбеддингов, используемой для векторизации текстов
SIZE = 250  # Размер фрагмента текста для разделения документов
OVERLAP = 50  # Перекрытие между фрагментами текста для обеспечения контекста

##Loader

In [ ]:
# Класс для загрузки документов из различных источников, поддерживающий работу с разными форматами файлов
class Loader:
    def load_single_document(self, file_path: str):
        # Метод для загрузки одного документа на основе пути к файлу
        ext = Path(file_path).suffix.lower()
        if ext in LOADER_MAPPING:
            loader_class, loader_args = LOADER_MAPPING[ext]
            loader_args['file_path'] = file_path;
            loader = loader_class(**loader_args)
            document = loader.load()
            return document
        else:
            raise ValueError(f"Unsupported file extension: {ext}")

    def load_documents(self, source_dir: str):
        # Метод для загрузки всех документов из указанной директории
        documents = []
        if os.path.isfile(source_dir):
          documents.extend(self.load_single_document(file_path))
        else:
          for root, _, files in os.walk(source_dir):
              for file_name in files:
                  file_path = os.path.join(root, file_name)
                  try:
                      document = self.load_single_document(file_path)
                      documents.extend(document)
                  except ValueError as e:
                      print(e)
        return documents

In [ ]:
loader = Loader()

example_document = loader.load_documents(path_to_documents)

example_document

NameError: name 'path_to_documents' is not defined

##Splitter

In [ ]:
# Класс для разделения документов на фрагменты определённого размера с заданным перекрытием
class Splitter:
    def __init__(self, chunk_size, chunk_overlap):
        # Инициализация параметров разделения: размер фрагмента и величина перекрытия
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap

    def split_documents(self, documents):
        fragments = []
        for document in documents:
            text = document.page_content
            doc_meta = document.metadata
            start = 0

            while start < len(text):
                end = min(start + self.chunk_size, len(text))
                fragment_text = text[start:end]

                fragment = {
                    "text": fragment_text,
                    "metadata": doc_meta
                }

                fragments.append(fragment)
                start += self.chunk_size - self.chunk_overlap

        return fragments


In [ ]:
splitter = Splitter(SIZE, OVERLAP)

example_fragments = splitter.split_documents(example_document)

example_fragments

[{'text': 'From: mathew <mathew@mantis.co.uk>\nSubject: Alt.Atheism FAQ: Atheist Resources\nSummary: Books, addresses, music -- anything related to atheism\nKeywords: FAQ, atheism, books, music, fiction, addresses, contacts\nExpires: Thu, 29 Apr 1993 11:57:19 GMT\nD',
  'metadata': {'source': '/content/drive/MyDrive/ITMO/ML/test_dir/49960.txt'}},
 {'text': ' contacts\nExpires: Thu, 29 Apr 1993 11:57:19 GMT\nDistribution: world\nOrganization: Mantis Consultants, Cambridge. UK.\nSupersedes: <19930301143317@mantis.co.uk>\nLines: 290\n\nArchive-name: atheism/resources\nAlt-atheism-archive-name: resources\nLast-modif',
  'metadata': {'source': '/content/drive/MyDrive/ITMO/ML/test_dir/49960.txt'}},
 {'text': 'ces\nAlt-atheism-archive-name: resources\nLast-modified: 11 December 1992\nVersion: 1.0\n\n                              Atheist Resources\n\n                      Addresses of Atheist Organizations\n\n                                     USA\n\nFREEDOM FROM RE',
  'metadata': {'source'

##Vector database

In [ ]:
from sentence_transformers import SentenceTransformer

# Базовый класс для создания эмбеддингов, обеспечивающий интерфейс для получения модели эмбеддингов
class Embedder:
    def __init__(self, model_name: str):
        # Инициализация эмбеддера
        self.model = SentenceTransformer(model_name)

    def get_embedder(self):
        # Метод для получения модели эмбеддингов, которая будет использоваться для векторизации текстов
        return self.model

    def encode(self, texts: list[dict]):
        return self.model.encode(texts)

In [ ]:
class HuggingFaceEmbedder(Embedder):
    def __init__(self):
        super().__init__(EMBEDDINGS)

In [ ]:
embedder = HuggingFaceEmbedder()

example_embedded_fragments = embedder.encode(example_fragments)

example_embedded_fragments

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


array([[-3.7023947e-02,  7.1858220e-02, -2.3285227e-03, ...,
         7.7936910e-02, -1.4454189e-01,  3.4529202e-02],
       [-4.5871064e-02,  1.7079403e-01, -1.9677655e-05, ...,
         5.0034132e-02, -4.1605800e-02, -6.1742783e-02],
       [-4.5468852e-02,  3.3188725e-01, -6.3532484e-03, ...,
         4.0434483e-02, -8.6400196e-02,  2.0218849e-02],
       ...,
       [-3.9947808e-02,  1.1800905e-01, -7.3795510e-03, ...,
         6.7709483e-02, -1.5134664e-02, -1.7877577e-02],
       [-1.6624042e-03,  2.0215838e-01,  5.1068771e-04, ...,
        -2.3176469e-02, -1.3295907e-01,  2.6410881e-02],
       [-3.3872660e-02, -7.9365365e-02, -7.0425342e-03, ...,
         6.0188740e-02, -1.2274976e-01,  9.6739188e-02]], dtype=float32)

### Базовые классы

In [ ]:
class Element():
  def __init__(self, embedding, metadata):
    self.embedding = embedding
    self.metadata = metadata

# Базовый класс для работы с коллекцией документов, поддерживающий добавление, поиск и очистку данных
class Collector:
    def __init__(self, splitter: Splitter, embedder: Embedder):
        self.splitter = splitter
        self.embedder = embedder
        self.loader = Loader()
        self.documents = []

    def add(self, texts: list[str], metadatas: list[dict]):
        # Метод для добавления текстов и связанных с ними метаданных в коллекцию
        added_documents = []
        embeddings = self.embedder.encode(texts)
        for embedding, metadata in zip(embeddings, metadatas):
            self.documents.append(Element(embedding=embedding, metadata=metadata))
            added_documents.append(Element(embedding=embedding, metadata=metadata))
        return added_documents

    def add_from_directory(self, dir_path: str):
        # Метод для добавления документов в коллекцию из указанной директории
        documents = self.loader.load_documents(dir_path)
        fragments = self.splitter.split_documents(documents)

        texts = [fragment["text"] for fragment in fragments]
        metadatas = [{"file_path": fragment["metadata"]} for fragment in fragments]

        self.add(texts, metadatas)

    def query_documents(self, embedding, top_k):
        pass

    def _cosine_similarity(self, vec1, vec2):
        dot_product = sum(a * b for a, b in zip(vec1, vec2))
        norm1 = sum(a ** 2 for a in vec1) ** 0.5
        norm2 = sum(b ** 2 for b in vec2) ** 0.5
        return dot_product / (norm1 * norm2)

    def get(self, search_strings: list[str], n_results: int) -> list[Document]:
        # Метод для поиска документов по строкам запроса с ограничением на количество результатов
        search_embeddings = self.embedder.encode(search_strings)
        results = []
        for search_embedding in search_embeddings:
            result_docs = self.query_documents(search_embedding, top_k=n_results)
            results.extend(result_docs)
        return results

    def get_documents(self, search_string: str, n_results: int, score_threshold: float) -> list[Document]:
        # Метод для поиска документов с учётом порога релевантности и количества возвращаемых результатов
        search_embedding = self.embedder.encode([search_string])[0]
        result_docs = self.query_documents(search_embedding, top_k=n_results)
        return [doc for doc in result_docs if doc.score >= score_threshold]

    def clear(self):
        # Метод для очистки коллекции документов
        self.documents.clear()

In [ ]:
class ChromaCollector(Collector):
    def __init__(self, splitter: Splitter, embedder: HuggingFaceEmbedder):
        super().__init__(splitter, embedder)
        self.client = Client()
        self.collection = self.client.get_or_create_collection(COLLECTION_NAME)
        self.doc_id_counter = 0

    def add(self, texts: list[str], metadatas: list[dict]):
        added_documents = super().add(texts, metadatas)
        print(added_documents)

        embeddings = [doc.embedding.tolist() for doc in added_documents]
        ids = [f"id_{self.doc_id_counter + i}" for i in range(len(added_documents))]
        self.doc_id_counter += len(added_documents)

        self.collection.add(
            documents=texts,
            embeddings=embeddings,
            metadatas=[doc.metadata for doc in added_documents],
            ids=ids
        )

    # def query_documents(self, embedding, top_k):
    #     return self.collection.query(embedding, top_k=top_k)

    def query_documents(self, embedding, top_k):
        documents_with_scores = [(doc, self._cosine_similarity(embedding, doc.embedding)) for doc in self.collection]
        documents_with_scores.sort(key=lambda x: x[1], reverse=True)
        return [doc for doc, score in documents_with_scores[:top_k]]

    def clear(self):
        super().clear()
        self.client.delete_collection(COLLECTION_NAME)

### Класс Collector

In [ ]:
import faiss

# Базовый класс для работы с коллекцией документов, поддерживающий добавление, поиск и очистку данных
class Collector:
    def __init__(self, splitter: Splitter, embedder: Embedder):
        self.splitter = splitter
        self.embedder = embedder
        self.loader = Loader()

    def add(self, texts: list[str], metadatas: list[dict]):
        # Метод для добавления текстов и связанных с ними метаданных в коллекцию
        embeddings = self.embedder.encode(texts)
        return [{"embedding": embedding, "metadata": metadata} for embedding, metadata in zip(embeddings, metadatas)]

    def add_from_directory(self, dir_path: str):
        # Метод для добавления документов в коллекцию из указанной директории
        documents = self.loader.load_documents(dir_path)
        fragments = self.splitter.split_documents(documents)

        texts = [fragment["text"] for fragment in fragments]
        metadatas = [fragment["metadata"] for fragment in fragments]
        self.add(texts, metadatas)

    def _cosine_similarity(self, vec1, vec2):
        dot_product = sum(a * b for a, b in zip(vec1, vec2))
        norm1 = sum(a ** 2 for a in vec1) ** 0.5
        norm2 = sum(b ** 2 for b in vec2) ** 0.5
        return dot_product / (norm1 * norm2)

    def query_documents(self, embedding, top_k):
        pass

    def get(self, search_strings: list[str], n_results: int) -> list[Document]:
        # Метод для поиска документов по строкам запроса с ограничением на количество результатов
        search_embeddings = self.embedder.encode(search_strings)
        results = []
        for search_embedding in search_embeddings:
            result_docs = self.query_documents(search_embedding, top_k=n_results)
            results.extend(result_docs)
        return results

    def get_documents(self, search_string: str, n_results: int, score_threshold: float) -> list[Document]:
        # Метод для поиска документов с учётом порога релевантности и количества возвращаемых результатов
        search_embedding = self.embedder.encode([search_string])[0]
        result_docs = self.query_documents(search_embedding, top_k=n_results)
        return [doc for doc in result_docs if doc.score >= score_threshold]


class ChromaCollector(Collector):
    def __init__(self, splitter: Splitter, embedder: HuggingFaceEmbedder):
        super().__init__(splitter, embedder)
        self.client = Client()
        self.collection = self.client.get_or_create_collection(COLLECTION_NAME)
        self.doc_id_counter = 0

    def add(self, texts: list[str], metadatas: list[dict]):
        # Добавляем документы с их эмбеддингами и метаданными в коллекцию
        added_documents = super().add(texts, metadatas)
        embeddings = np.array([doc["embedding"] for doc in added_documents]).astype("float32")

        ids = [f"id_{self.doc_id_counter + i}" for i in range(len(added_documents))]
        self.doc_id_counter += len(added_documents)

        self.collection.add(
            documents=texts,
            embeddings=embeddings.tolist(),
            metadatas=[doc["metadata"] for doc in added_documents],
            ids=ids
        )

    def query_documents(self, query_embedding, top_k: int):
        results = self.collection.query(
            query_embeddings=[query_embedding.tolist()],
            n_results=top_k
        )

        ids = results['ids'][0]
        distances = results['distances'][0]
        metadatas = results['metadatas'][0]
        documents = results['documents'][0]

        results_list = []
        for i in range(len(ids)):
            results_list.append({
                "id": ids[i],
                "distance": distances[i],
                "metadata": metadatas[i],
                "document": documents[i]
            })

        return results_list


    def clear(self):
        self.client.delete_collection(COLLECTION_NAME)

###Implementation vector database

In [ ]:
# path_to_index = '/content/VDB' #@param {type:"string"}
# path_to_documents = '/content/tmp' #@param {type:"string"}

In [ ]:
#Нужно написать реализацию векторной базы данных

##Search

In [ ]:
query = 'How can I clean a suede jacket?' #@param {type:"string"}
n_results = 5 #@param {type:"integer"}
score_threshold = 0.5 # @param {type:"slider", min:0, max:1, step:0.1}


In [ ]:
#Нужно реализовать эксперимент по поиску в векторном индексе
splitter = Splitter(SIZE, OVERLAP)
embedder = HuggingFaceEmbedder()
collector = ChromaCollector(splitter, embedder)

collector.clear()
collector = ChromaCollector(splitter, embedder)

documents = [
        "The sun is an average star.",
        "The moon orbits the Earth.",
        "I enjoy cooking and baking.",
        "The galaxy is vast and full of stars.",
        "Artificial intelligence is transforming industries."
    ]

metadatas = [
    {'file_path': 'dir_1'},
    {'file_path': 'dir_2'},
    {'file_path': 'dir_3'},
    {'file_path': 'dir_4'},
    {'file_path': 'dir_5'},
    ]


collector.add(documents, metadatas)
collector.collection.peek()

query = "What is sun?"
query_embedding = embedder.encode([query])[0]
collector.query_documents(query_embedding, 2)

[{'id': 'id_0',
  'distance': 8.915149688720703,
  'metadata': {'file_path': 'dir_1'},
  'document': 'The sun is an average star.'},
 {'id': 'id_3',
  'distance': 14.085482597351074,
  'metadata': {'file_path': 'dir_4'},
  'document': 'The galaxy is vast and full of stars.'}]

In [ ]:
splitter = Splitter(SIZE, OVERLAP)
embedder = HuggingFaceEmbedder()
collector = ChromaCollector(splitter, embedder)

collector.clear()
collector = ChromaCollector(splitter, embedder)

In [ ]:
path_to_documents = '/content/drive/MyDrive/ITMO/ML/dataset/20news-bydate-test/comp.graphics'

collector.add_from_directory(path_to_documents)

In [ ]:
collector.collection.peek(500)

{'ids': ['id_0',
  'id_1',
  'id_2',
  'id_3',
  'id_4',
  'id_5',
  'id_6',
  'id_7',
  'id_8',
  'id_9',
  'id_10',
  'id_11',
  'id_12',
  'id_13',
  'id_14',
  'id_15',
  'id_16',
  'id_17',
  'id_18',
  'id_19',
  'id_20',
  'id_21',
  'id_22',
  'id_23',
  'id_24',
  'id_25',
  'id_26',
  'id_27',
  'id_28',
  'id_29',
  'id_30',
  'id_31',
  'id_32',
  'id_33',
  'id_34',
  'id_35',
  'id_36',
  'id_37',
  'id_38',
  'id_39',
  'id_40',
  'id_41',
  'id_42',
  'id_43',
  'id_44',
  'id_45',
  'id_46',
  'id_47',
  'id_48',
  'id_49',
  'id_50',
  'id_51',
  'id_52',
  'id_53',
  'id_54',
  'id_55',
  'id_56',
  'id_57',
  'id_58',
  'id_59',
  'id_60',
  'id_61',
  'id_62',
  'id_63',
  'id_64',
  'id_65',
  'id_66',
  'id_67',
  'id_68',
  'id_69',
  'id_70',
  'id_71',
  'id_72',
  'id_73',
  'id_74',
  'id_75',
  'id_76',
  'id_77',
  'id_78',
  'id_79',
  'id_80',
  'id_81',
  'id_82',
  'id_83',
  'id_84',
  'id_85',
  'id_86',
  'id_87',
  'id_88',
  'id_89',
  'id_90',
  

In [ ]:
search_queries = ["What tools does include ImageMagick?"]

for query in search_queries:
    print(f"\nSearch query: {query}")
    results = collector.get([query], n_results=n_results)

    for i, doc in enumerate(results):
        print(f"Result {i + 1}: {doc}")


Search query: What tools does include ImageMagick?
Result 1: {'id': 'id_99', 'distance': 2.390554904937744, 'metadata': {'source': '/content/drive/MyDrive/ITMO/ML/dataset/20news-bydate-test/comp.graphics/38778.txt'}, 'document': '   ImageMagick, an X11 package for display and interactive manipulation\n    of images.  Includes tools for image conversion, annotation, compositing,\n    animation, and creating montages.  ImageMagick can read and write many of\n    the more popular i'}
Result 2: {'id': 'id_2707', 'distance': 4.0286030769348145, 'metadata': {'source': '/content/drive/MyDrive/ITMO/ML/dataset/20news-bydate-test/comp.graphics/39078.txt'}, 'document': 'as.  ImageMaster, from Black Belt Systems, is another\nwell-regarded commercial graphics package with JPEG support.\n\nThe free IJG JPEG software is available compiled for Amigas from\namiga.physik.unizh.ch (and mirror sites) in directory amiga/gfx/conv,'}
Result 3: {'id': 'id_3437', 'distance': 4.581252574920654, 'metadata': {'s

##Evaluation

In [ ]:
# Класс для оценки работы коллектора, предоставляющий функционал для поиска, оценки и расчета статистики по результатам
class CollectorEvaluator:
    def __init__(self, collector: Collector, n_top=5):
        # Инициализация коллектора и параметра n_top для ограничения числа возвращаемых результатов
        self.collector = collector
        self.n_top = n_top

    def explore_collector(self, text: str):
        # Метод для поиска документов в коллекторе на основе текста запроса
        collector_results = collector.get([text], n_results=self.n_top)
        return collector_results

    def eval(self, query: str, answer: str):
        # Метод для оценки корректности найденных документов на основе запроса и правильного ответа
        collector_results = self.explore_collector(query)

        print(f"\nSearch query: {query},\nanswer: {answer}")

        for i, doc in enumerate(collector_results, start=1):
            print(f"Result {i}: {doc}")

        for i, doc in enumerate(collector_results, start=1):
            if answer in doc["document"]:
                return i
        return None

    def calculate_statistics(self, data: list[int]):
        # Метод для расчета статистических показателей (например, минимальное, максимальное, среднее значение)

        filtered_data = [serial_number for serial_number in data if serial_number is not None]
        not_found_queries = len([serial_number for serial_number in data if serial_number is None])

        if not filtered_data:
            return {"min": None, "max": None, "mean": None, "not found": None}

        min_serial_number = min(filtered_data)
        max_serial_number = max(filtered_data)
        mean_serial_number = sum(filtered_data) / len(filtered_data)
        return {"min": min_serial_number, "max": max_serial_number, "mean": mean_serial_number, "not_found": not_found_queries / len(data)}

    def explore_and_calculate(self, data: list[tuple[str, str]]):
        # Метод для проведения поиска по данным и расчета статистики на основе результатов
        collector_results = []
        for query, answer in data:
            serial_number = self.eval(query, answer)
            collector_results.append(serial_number)

        stats = self.calculate_statistics(collector_results)
        return stats

In [ ]:
path_to_dataset = '/content/QA.csv' #@param {type:"string"}
n_lines = 100 #@param {type:"integer"}
n_top = 10 #@param {type:"integer"}



In [ ]:
collector.clear()

In [ ]:
#Нужно написать эксперимент для оценки полученной коллекции
collector = ChromaCollector(splitter, embedder)
evaluator = CollectorEvaluator(collector, n_top=n_top)

data = [
    ("What is the sun?", "The sun is an average star."),
    ("What is the moon?", "The moon orbits the Earth."),
    ("Cooking tips", "I enjoy cooking and baking.")
]

stats = evaluator.explore_and_calculate(data)
print("Статистика по позициям релевантных ответов:", stats)



Search query: What is the sun?,
 answer: The sun is an average star.
Result 1: {'id': 'id_629', 'distance': 10.744452476501465, 'metadata': {'source': '/content/drive/MyDrive/ITMO/ML/dataset/20news-bydate-test/comp.graphics/38847.txt'}, 'document': 'he Parallax one).\n\nThanks in advance.\n\nGilles\n\nPS:  We are using a Sun Sparc running X11.\n'}
Result 2: {'id': 'id_418', 'distance': 10.759300231933594, 'metadata': {'source': '/content/drive/MyDrive/ITMO/ML/dataset/20news-bydate-test/comp.graphics/38852.txt'}, 'document': 'ources.sun on 11dec89.  Also available via email to\n  alv-users-request@cs.bris.ac.uk.\n\nAIPS\n----\n  Astronomical Image Processing System.  Contact: aipsmail@nrao.edu\n  (also see the UseNet Newsgroups alt.sci.astro.aips and sci.astro.fits)\n  Built by '}
Result 3: {'id': 'id_432', 'distance': 10.804450988769531, 'metadata': {'source': '/content/drive/MyDrive/ITMO/ML/dataset/20news-bydate-test/comp.graphics/38852.txt'}, 'document': "n be obtained from\n  Keith

In [ ]:
data = [
    ('What are some examples of toolkits that can be used for image format conversion and basic image manipulations?', 'umber of toolkits for converting from one image format to\nanother, doing simple image manipulations such as size scaling, plus\nthe above-mentioned 24 -> 8, color -> gray, gray -> b&w conversions.\nHere are pointers to some of them:\n\n    xv by John Bra'),
    ('What techniques are discussed in the context of quantizing 24-bit images down to 8 bits, and where can one find a relevant reference on this topic?', 'for\nshading, chapter 19 for clipping, and branch out from there.\n\n\n3) Quantizing 24 bit images down to 8 bits.\n\nFind a copy of "Color Image Quantization for Frame Buffer Display" by\nPaul Heckbert, SIGGRAPH \'82 Proceedings, page 297.  There are other\n'),
    ('How to FTP by email', ' 9) Converting between vector formats.\n    10) How to get Pixar films.\n    11) How do I draw a circle as a Bezier (or B-spline) curve?\n    12) How to order standards documents.\n    13) How to FTP by email.\n    14) How to tell whether a point is withi'),
    ('What steps should you take to obtain information about using the mail handler and software distribution?', ' exercises.  To receive information describing\nhow you can use the mail handler, simply mail graphtext@cs.brown.edu\nand put the word "Help" in the Subject line.  Use the Subject line\n"Software-Distribution" to receive information specifically concern'),
    ('How to join ACM/SIGGRAPH\n', 'trace height fields\n    24) How to find the area of a 3D polygon\n    25) How to join ACM/SIGGRAPH\n    26) Where can I find MRI and CT scan volume data?\n    27) Specific references on spatial data structures including quadtrees\n\tand octrees\n    28) Wh'),
    ('How to get general information about the\nmail server?', '/news.answers/pictures-faq/part1\nsend usenet/news.answers/pictures-faq/part2\n\nSend a message containing "help" to get general information about the\nmail server.\n\nAlso, you could check out the resources described in sections 7, 8, and\n20 above for mor'),
    ('How many tool the kit contains on image manipulation, digital halftoning?', 'rting pixels of arbitrary channels,\n    components, and bit precisions while allowing compression and machine\n    byte-order independence.  The kit contains more than 50 tools with\n    extensive support of image manipulation, digital halftoning and f'),
    ('A Fast Algorithm for Raster\nRotation', 'implementation is\nalso present in PBMPLUS.  Reference: "A Fast Algorithm for Raster\nRotation", by Alan Paeth (awpaeth@watcgl.waterloo.edu) Graphics\nInterface \'86 (Vancouver).  An article on the IM toolkit appears in\nthe same journal.  An updated vers'),
    ('What are some examples of formats that can be converted or rendered by commercial PostScript clones for PCs?', " to Sun raster format, or HPGL to\nX11 bitmap.  For example, some of the commercial PostScript clones for\nPC's allow you to render to a disk file as well as a printer.  Also,\nthe PostScript interpreters in the NeXT box and in Sun's X11/NeWs can\nbe use"),
    ('Why is assembly language used for over 100 functions in the graphical interface?', "short or floating point arithmetic to maintain the precision\n  and accuracy of the pixel format. Over 100 functions are hand-coded in\n  assembly language for maximum speed on the Intel hardware.  The entire\n  graphical interface is also written in as")
]

In [ ]:
#Нужно написать эксперимент для оценки полученной коллекции
collector = ChromaCollector(splitter, embedder)
evaluator = CollectorEvaluator(collector, n_top=n_top)

stats = evaluator.explore_and_calculate(data)
print("Статистика по позициям релевантных ответов:", stats)


Search query: What are some examples of toolkits that can be used for image format conversion and basic image manipulations?,
answer: umber of toolkits for converting from one image format to
another, doing simple image manipulations such as size scaling, plus
the above-mentioned 24 -> 8, color -> gray, gray -> b&w conversions.
Here are pointers to some of them:

    xv by John Bra
Result 1: {'id': 'id_78', 'distance': 2.7567696571350098, 'metadata': {'source': '/content/drive/MyDrive/ITMO/ML/dataset/20news-bydate-test/comp.graphics/38778.txt'}, 'document': 'umber of toolkits for converting from one image format to\nanother, doing simple image manipulations such as size scaling, plus\nthe above-mentioned 24 -> 8, color -> gray, gray -> b&w conversions.\nHere are pointers to some of them:\n\n    xv by John Bra'}
Result 2: {'id': 'id_114', 'distance': 3.634584426879883, 'metadata': {'source': '/content/drive/MyDrive/ITMO/ML/dataset/20news-bydate-test/comp.graphics/38778.txt'}, 'document